In [19]:
using Plots
using LinearAlgebra
using SpecialFunctions

In [20]:
#exponents = [3.425250914, 0.6239137298, 0.1688554040]


# Helpful structs
struct Gaussian
    exponent::Number
    center::AbstractArray{<:Number, 1}
end


struct Atom
    charge::Number
    position::AbstractArray{<:Number, 1}
end


struct HFObject
    C::AbstractArray{<:Number,2}
    P::AbstractArray{<:Number,2}
    H::AbstractArray{<:Number,2}
    F::AbstractArray{<:Number,2}
end
#############

# Integral functions
function overlap_integral(A::T, B::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    
    return (4*α*β / (α + β) ^ 2) ^ (3/4) * exp(-α*β / (α + β) * norm(R_A - R_B)^2)
end

function overlap_integral(arr::Tuple{T, T}) where T <: Gaussian
    return overlap_integral(arr[1], arr[2])
end


function kinetic_integral(A::T, B::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    
    a = α * β / (α + β)
    
    return overlap_integral(A, B) * a * (3 - 2 * a * norm(R_A - R_B)^2)
end

function kinetic_integral(arr::Tuple{T, T}) where T <: Gaussian
    return kinetic_integral(arr[1], arr[2])
end


function core_potential_integral(A::T, B::T, C::Atom) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    R_A = A.center
    R_B = B.center
    R_C = C.position
    Z = C.charge
        
    b = sqrt(α + β)
    
    R_P = (α * R_A + β * R_B) / (α + β)
    
    if R_P == R_C
        return -overlap_integral(A, B) * Z * 2 * b / sqrt(π)
    else
        return -overlap_integral(A, B) * Z / norm(R_P - R_C) * erf(b * norm(R_P - R_C))
    end
end

function core_potential_integral(A::T, B::T, atoms::AbstractArray{Atom,1}) where T<:Gaussian
    return sum(core_potential_integral(A, B, a) for a in atoms)
end

function core_potential_integral(arr::Tuple{T, T}, atoms::AbstractArray{Atom,1}) where T<:Gaussian
    return core_potential_integral(arr[1], arr[2], atoms)
end


function electron_electron_potential(A::T, B::T, C::T, D::T) where T<:Gaussian
    α = A.exponent
    β = B.exponent
    γ = C.exponent
    δ = D.exponent
    
    R_A = A.center
    R_B = B.center
    R_C = C.center
    R_D = D.center
    
    R_P = (α * R_A + β * R_B) / (α + β)
    R_Q = (γ * R_C + δ * R_D) / (γ + δ)
    
    c = sqrt((α + β) * (γ + δ) / (α + β + γ + δ))
    
    if R_P == R_Q 
        return overlap_integral(A, B) * overlap_integral(C, D) * 2 * c / sqrt(π)
    else
        return overlap_integral(A, B) * overlap_integral(C, D) / norm(R_P - R_Q) * erf(c * norm(R_P - R_Q))
    end
end

function electron_electron_potential(arr::NTuple{4, Gaussian})
    return electron_electron_potential(arr[1], arr[2], arr[3], arr[4])
end


function exchange_integral(A::T, B::T, C::T, D::T) where T<:Gaussian
    return electron_electron_potential(A, C, B, D)
end

function exchange_integral(arr::NTuple{4, Gaussian})
    return exchange_integral(arr[1], arr[2], arr[3], arr[4])
end

#################

exchange_integral (generic function with 2 methods)

In [33]:
# Reading molecular info from file


function read_mol_settings(filename)
    positions = []
    exponents = []
    
    open(filename, "r") do mol_file
        while !eof(mol_file)
            line = readline(mol_file)
            if line == "COORDINATES:"
                while line != "==="
                    line = readline(mol_file)
                    if line == "==="
                        break
                    end
                    push!(positions, map(x->parse(Float64, x), split(line))) 
                end
            end
            if line == "EXPONENTS:"
                while line != "==="
                    line = readline(mol_file)
                    if line == "==="
                        break
                    end
                    push!(exponents, tryparse(Float64, line)) 
                end
            end
            if line == "ELECTRONS:"
                line = readline(mol_file)
                global N = tryparse(Int64, line)
            end
        end
    end

    atoms = Array{Atom}(undef, length(positions))
    gaussians = Array{Gaussian}(undef, length(positions) * length(exponents))

    n = 1
    for e in exponents
        for p in positions
            if n <= length(positions)
                atoms[n] = Atom(1, p)
            end
            gaussians[n] = Gaussian(e, p)
            n += 1
        end
    end
    return atoms, gaussians
end

read_mol_settings (generic function with 1 method)

In [34]:
atoms, gaussians = read_mol_settings("H2.txt")
K = length(gaussians)

4

In [35]:
function fock_matrix(P)
    one_body = Iterators.product(gaussians, gaussians)
    S = map(x->overlap_integral(x), one_body)
    T = map(x->kinetic_integral(x), one_body)
    V = map(x->core_potential_integral(x, atoms), one_body)

    H = T + V

    two_body = Iterators.product(gaussians, gaussians, gaussians, gaussians)
    M0 = map(x->electron_electron_potential(x), two_body)
    M1 = map(x->exchange_integral(x), two_body)
    M = M0 - 1/2 * M1

    M_reshaped = reshape(M, K^2, K^2) # have to reshape for multiplication
    G = reshape(M_reshaped * P[:], K, K)
    F = H + G
    
    return F
end

function electron_energy(P)
    one_body = Iterators.product(gaussians, gaussians)
    S = map(x->overlap_integral(x), one_body)
    T = map(x->kinetic_integral(x), one_body)
    V = map(x->core_potential_integral(x, atoms), one_body)

    H = T + V

    two_body = Iterators.product(gaussians, gaussians, gaussians, gaussians)
    M0 = map(x->electron_electron_potential(x), two_body)
    M1 = map(x->exchange_integral(x), two_body)
    M = M0 - 1/2 * M1

    M_reshaped = reshape(M, K^2, K^2) # have to reshape for multiplication
    G = reshape(M_reshaped * P[:], K, K)
    F = H + G

    E = 1/2 * sum(P .* (H + F))
    
    return E
end

function nuclear_repulsion_energy(A::Atom, B::Atom)
    E = A.charge * B.charge / norm(A.position - B.position)
    return E
end

function nuclear_repulsion_energy(atoms::AbstractArray{Atom})
    E = 0
    for (i, a1) in enumerate(atoms[1:end-1])
        for a2 in atoms[i+1:end]
            E += nuclear_repulsion_energy(a1, a2)
        end
    end
    
    return E
end

function total_energy(P)
    E_tot = electron_energy(P) + nuclear_repulsion_energy(atoms)
    return E_tot
end

total_energy (generic function with 1 method)

In [36]:
# Assemble matrices

function hf_step(P)

    one_body = Iterators.product(gaussians, gaussians)
    S = map(x->overlap_integral(x), one_body)
    T = map(x->kinetic_integral(x), one_body)
    V = map(x->core_potential_integral(x, atoms), one_body)

    H = T + V

    two_body = Iterators.product(gaussians, gaussians, gaussians, gaussians)
    M0 = map(x->electron_electron_potential(x), two_body)
    M1 = map(x->exchange_integral(x), two_body)
    M = M0 - 1/2 * M1

    M_reshaped = reshape(M, K^2, K^2) # have to reshape for multiplication
    G = reshape(M_reshaped * P[:], K, K)
    F = H + G

    X = S^(-1/2)
    F_dash = Symmetric(X * F * X)

    eigen_obj = eigen(F_dash)

    E, B = eigen_obj # These are sorted by default
    C = X * B
    
    P_new = 2 * C[:, 1:N ÷ 2] * C[:, 1:N ÷ 2]'
    return HFObject(C,P_new,H,F)
end


function converge_hf(tol::Number=1e-6)
    P_old = Matrix(I,K,K)
    diff = Inf
    hf_return = undef
    iterations = 0
    while diff > tol
        iterations += 1
        hf_return = hf_step(P_old)
        global P_new = hf_return.P
        diff = norm(P_new - P_old)
        P_old = P_new
    end
    println("Converged after $iterations iterations")
    return hf_return
end

converge_hf (generic function with 2 methods)

In [37]:
final_hf_object = converge_hf(1e-12)
final_hf_object.C

Converged after 16 iterations


4×4 Array{Float64,2}:
 -0.39309    1.46452    -0.595576   1.04801 
 -0.39309   -1.46452    -0.595576  -1.04801 
 -0.209723   0.0840984   0.821362  -0.957419
 -0.209723  -0.0840984   0.821362   0.957419

In [38]:
total_energy(final_hf_object.P)

-1.0966398387448086

In [27]:
println("Core potential integrals")
for g1 in gaussians
    for g2 in gaussians
        V = sum(core_potential_integral(g1, g2, a) for a in atoms)
        print(V, " ")
    end
    println()
end

println("\nKinetic integrals")
for g1 in gaussians
    for g2 in gaussians
        T = kinetic_integral(g1, g2)
        print(T, " ")
    end
end

println("\nElectron-electron integrals")
for (i, g1) in enumerate(gaussians)
    for (j, g2) in enumerate(gaussians)
        for (k, g3) in enumerate(gaussians)
            for (l, g4) in enumerate(gaussians)
                M0 = electron_electron_potential(g1, g2, g3, g4)
                M1 = exchange_integral(g1, g2, g3, g4)
                println("($(i)$(j)|$(k)$(l)) ", M0 - 1/2 * M1, " ")
            end
        end
    end
end

Core potential integrals
-2.1264644329454763 -1.69796857792814 -2.1225030258440563 -1.2148704674323794 

UndefRefError: UndefRefError: access to undefined reference